# Advanced Scoring of Graded Readers
## DS 7337 - Natural Language Processing - Homework 2
### George C. Sturrock
### January 26, 2019
#### Homework 2 Objectives
1.	In Python, create a method for scoring the vocabulary size of a text, and normalize the score from 0 to 1. It does not matter what method you use for normalization as long as you explain it in a short paragraph. 
2.	After consulting section 3.2 in chapter 1 of Bird-Klein, create a method for scoring the long-word vocabulary size of a text, and likewise normalize (and explain) the scoring as in step 1 above.
3.	Now create a “text difficulty score” by combining the lexical diversity score from homework 1, and your normalized score of vocabulary size and long-word vocabulary size, in equal weighting. Explain what you see when this score is applied to same graded texts used in homework 1.

#### Import Libraries

In [66]:
import platform; print("Platform:", platform.platform())
import os
import sys; print("Python:", sys.version)
import requests; print("Requests:", requests.__version__)

import nltk; print("NLTK Version:", nltk.__version__)
from nltk.corpus import stopwords
theStopWords = set(stopwords.words('english'))

import numpy as np; print("Numpy Version:", np.__version__)
import pandas as pd; print("Pandas Version:", pd.__version__)
import matplotlib.pyplot as plt; #print("Matplotlib Pyplot Version:", plt.__version__)
import string; #print("String Version:", string.__version__)
from langdetect import detect; print("langdetect Version: 1.0.7")

import seaborn as sns; print("Seaborn Version:", sns.__version__)
sns.set(style='darkgrid')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup; print("BeatifulSoup Version: 4.6.3")

Platform: Linux-4.15.0-1027-gcp-x86_64-with-debian-stretch-sid
Python: 3.5.6 |Anaconda custom (64-bit)| (default, Aug 26 2018, 21:41:56) 
[GCC 7.3.0]
Requests: 2.19.1
NLTK Version: 3.3
Numpy Version: 1.15.2
Pandas Version: 0.23.4
langdetect Version: 1.0.7
Seaborn Version: 0.9.0
BeatifulSoup Version: 4.6.3


##### To Do:
1.  Remove licensing
2.  find correct a hrefs and automatically build list of links and download books
3.  Navigate to .txt files for each book

#### Create NLP Pipeline Function

def nlp_pipeline(url):
    #Download source web page, strip HTML if necessary, trim to desired content
    html = urlopen(url[0]).read()
    html = html.decode("utf-8")
    raw = BeautifulSoup(html, "html.parser").get_text(strip=True).lower()
    position, raw_text_without_license = remove_gutenberg_license(raw)
    
    #tokenize the text including the header information
    tokens = nltk.work_tokenize

In [53]:
#Open one text
from urllib import request

#url = "http://www.gutenberg.org/cache/epub/15659/pg15659.txt" #The beacon second reader
#url = "http://www.gutenberg.org/files/22917/22917-0.txt" #french book
#url = "http://www.gutenberg.org/cache/epub/15353/pg15353.txt" #spanish book
#url = "http://www.gutenberg.org/cache/epub/24072/pg24072.txt" #Italian
url = 'http://www.gutenberg.org/cache/epub/8997/pg8997.txt' #latin
html = request.urlopen(url).read().decode('utf8')

In [54]:
#print contents of html
print(type(html))
print(html)

<class 'str'>
﻿The Project Gutenberg EBook of Ritchie's Fabulae Faciles, by Various

This eBook is for the use of anyone anywhere in the United States and most
other parts of the world at no cost and with almost no restrictions
whatsoever.  You may copy it, give it away or re-use it under the terms of
the Project Gutenberg License included with this eBook or online at
www.gutenberg.org.  If you are not located in the United States, you'll have
to check the laws of the country where you are located before using this ebook.

Title: Ritchie's Fabulae Faciles
       A First Latin Reader

Author: Various

Editor: John Kirtland

Posting Date: February 24, 2015 [EBook #8997]
Release Date: September, 2005
First Posted: August 31, 2003
Last Updated: October 14, 2017

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK RITCHIE'S FABULAE FACILES ***




Produced by Karl Hagen, Tapio Riikonen and Online
Distributed Proofreaders










RITCHIE'S FABULAE FACILES

A FIRST LATIN READER




In [55]:
print(html.splitlines())

["\ufeffThe Project Gutenberg EBook of Ritchie's Fabulae Faciles, by Various", '', 'This eBook is for the use of anyone anywhere in the United States and most', 'other parts of the world at no cost and with almost no restrictions', 'whatsoever.  You may copy it, give it away or re-use it under the terms of', 'the Project Gutenberg License included with this eBook or online at', "www.gutenberg.org.  If you are not located in the United States, you'll have", 'to check the laws of the country where you are located before using this ebook.', '', "Title: Ritchie's Fabulae Faciles", '       A First Latin Reader', '', 'Author: Various', '', 'Editor: John Kirtland', '', 'Posting Date: February 24, 2015 [EBook #8997]', 'Release Date: September, 2005', 'First Posted: August 31, 2003', 'Last Updated: October 14, 2017', '', 'Language: English', '', '', "*** START OF THIS PROJECT GUTENBERG EBOOK RITCHIE'S FABULAE FACILES ***", '', '', '', '', 'Produced by Karl Hagen, Tapio Riikonen and Online', 'Di

In [56]:
#split line of html and remove licensing
## credit to: https://stackoverflow.com/questions/20461847/str-startswith-with-a-list-of-strings-to-test-for
html2 = html.splitlines()
print("Text Title:", html2[0])
#print(html2.index("*** START OF THIS PROJECT GUTENBERG EBOOK THE BEACON SECOND READER ***")) #no match returns ValueError
start = [i for i, s in enumerate(html2) if s.startswith("*** START OF THIS PROJECT GUTENBERG EBOOK")]
#start = [i for i, s in enumerate(html2) if s.startswith("[Illustration")]
end = [i for i, s in enumerate(html2) if s.startswith("*** END OF THIS PROJECT GUTENBERG EBOOK")]
print(start[0], end[0])
text = html2[(start[0]+1):(end[0]-2)]
print("Type:", type(text), "Length:", len(text))
print(text)

Text Title: ﻿The Project Gutenberg EBook of Ritchie's Fabulae Faciles, by Various
24 5594
Type: <class 'list'> Length: 5567
['', '', '', '', 'Produced by Karl Hagen, Tapio Riikonen and Online', 'Distributed Proofreaders', '', '', '', '', '', '', '', '', '', '', "RITCHIE'S FABULAE FACILES", '', 'A FIRST LATIN READER', '', '', '_EDITED WITH NOTES AND A VOCABULARY_', '', 'BY', 'JOHN COPELAND KIRTLAND, Jr.', '_Professor of Latin in The Phillips Exeter Academy_', '', '', '', 'THE LITTLE THAT IS MINE IN THIS LITTLE BOOK I GRATEFULLY DEDICATE TO', "PROFESSOR JOSEPH HETHERINGTON M'DANIELS TEACHER AND FRIEND", '', '', '', '', 'PREFACE', '', '', 'Some time ago a fellow-teacher brought the _Fabulae Faciles_ to my', 'notice, and I have since used two of them each year with my class of', 'beginners in Latin with increasing appreciation. Indeed, I know nothing', 'better to introduce the student into the reading of connected narrative,', "and to bridge the great gulf between the beginner's book of th

In [57]:
len(str(text))

253866

In [58]:
type(start)
#type(end)

list

In [59]:
start

[24]

In [60]:
if start == []:
    print('yes')
else:
    print('no')

no


In [61]:
#Strip New Line Characters and convert to lower case
strip = list()

for t in text:
    strip.append(t.rstrip().lower())

print(type(strip))
print(str(strip))

<class 'list'>
['', '', '', '', 'produced by karl hagen, tapio riikonen and online', 'distributed proofreaders', '', '', '', '', '', '', '', '', '', '', "ritchie's fabulae faciles", '', 'a first latin reader', '', '', '_edited with notes and a vocabulary_', '', 'by', 'john copeland kirtland, jr.', '_professor of latin in the phillips exeter academy_', '', '', '', 'the little that is mine in this little book i gratefully dedicate to', "professor joseph hetherington m'daniels teacher and friend", '', '', '', '', 'preface', '', '', 'some time ago a fellow-teacher brought the _fabulae faciles_ to my', 'notice, and i have since used two of them each year with my class of', 'beginners in latin with increasing appreciation. indeed, i know nothing', 'better to introduce the student into the reading of connected narrative,', "and to bridge the great gulf between the beginner's book of the", 'prevailing type and the latinity of caesar or nepos. they are adapted to', 'this use not merely by reaso

In [62]:
table = str.maketrans('', '', string.punctuation)

#tokenize text
raw = nltk.word_tokenize(str(strip))
raw2 = [r.translate(table) for r in raw]
rawx = [rx for rx in raw2 if rx not in theStopWords and rx.isalpha()]
longword = 0
#for r in rawx:
#    if len(r) > 
raw3 = nltk.Text(rawx)
raw4 = raw3.tokens
raw4

['produced',
 'karl',
 'hagen',
 'tapio',
 'riikonen',
 'online',
 'distributed',
 'proofreaders',
 'ritchie',
 'fabulae',
 'faciles',
 'first',
 'latin',
 'reader',
 'edited',
 'notes',
 'vocabulary',
 'john',
 'copeland',
 'kirtland',
 'jr',
 'professor',
 'latin',
 'phillips',
 'exeter',
 'academy',
 'little',
 'mine',
 'little',
 'book',
 'gratefully',
 'dedicate',
 'professor',
 'joseph',
 'hetherington',
 'mdaniels',
 'teacher',
 'friend',
 'preface',
 'time',
 'ago',
 'fellowteacher',
 'brought',
 'fabulae',
 'faciles',
 'notice',
 'since',
 'used',
 'two',
 'year',
 'class',
 'beginners',
 'latin',
 'increasing',
 'appreciation',
 'indeed',
 'know',
 'nothing',
 'better',
 'introduce',
 'student',
 'reading',
 'connected',
 'narrative',
 'bridge',
 'great',
 'gulf',
 'beginner',
 'book',
 'prevailing',
 'type',
 'latinity',
 'caesar',
 'nepos',
 'adapted',
 'use',
 'merely',
 'reason',
 'simplicity',
 'interest',
 'particularly',
 'graduating',
 'difficulties',
 'large',
 'use'

In [63]:
#'fr' = french
#'it' = italian
#'es' = spanish
#'en' = english
lang = detect(''.join(raw2))
lang

'en'

In [ ]:
lang.

In [21]:
type(raw4)

list

In [80]:
#remove stopwords, blanks and other 
words = []

for r in raw4:
    if r not in theStopWords:
        if r not in '':
            if r.isalpha():
                words.append(r)

                
print(type(words))
print(len(words))
print(words)
word2 = str(words)
tokens = nltk.word_tokenize(word2)
print("==========================================================================")
print("==========================================================================")
print(type(tokens))
print(len(tokens))
print(tokens)


<class 'list'>
11430
['page', 'ship', 'old', 'english', 'rhyme', 'wolf', 'seven', 'young', 'kids', 'william', 'jacob', 'grimm', 'nt', 'think', 'phoebe', 'cary', 'tom', 'thumb', 'english', 'fairy', 'tale', 'suppose', 'alice', 'cary', 'cinderella', 'english', 'fairy', 'tale', 'raindrops', 'ann', 'hawkshawe', 'four', 'friends', 'william', 'jacob', 'grimm', 'little', 'birdie', 'alfred', 'tennyson', 'mother', 'frost', 'william', 'jacob', 'grimm', 'ever', 'see', 'lydia', 'maria', 'child', 'bear', 'tail', 'short', 'german', 'folk', 'tale', 'rumpelstiltskin', 'william', 'jacob', 'grimm', 'bed', 'summer', 'robert', 'louis', 'stevenson', 'golden', 'touch', 'greek', 'myth', 'meadow', 'olive', 'wadsworth', 'bell', 'atri', 'german', 'folk', 'tale', 'baby', 'hugh', 'miller', 'bruce', 'spider', 'scottish', 'tradition', 'wise', 'little', 'pig', 'anonymous', 'indian', 'story', 'good', 'play', 'robert', 'louis', 'stevenson', 'dick', 'whittington', 'english', 'folk', 'tale', 'new', 'moon', 'eliza', 'lee'

### Saved Code

In [9]:
### Turn web page to text ###
try:
    del raw, raw2, raw3, raw4, raw5
except Exception as e:
    print(e)

table = str.maketrans('', '', string.punctuation)

#get text out of HTML
raw = BeautifulSoup(html).get_text()
raw2 = nltk.word_tokenize(raw)
raw3 = [r.translate(table) for r in raw2]
raw4 = nltk.Text(raw3)
raw5 = raw4.tokens

text = []

#remove stopwords, blanks and other 
for r in raw5:
    if r not in theStopWords:
        if r not in '':
            if r.isalpha():
                text.append(r)
            
print(text)
print(len(text))

name 'raw6' is not defined
['icon', 'background', 'transparent', 'url', 'picsspritepng', 'norepeat', 'CDATA', 'var', 'jsonsearch', 'ebookssuggest', 'var', 'mobileurl', 'var', 'canonicalurl', 'http', 'var', 'lang', 'enUS', 'var', 'fblang', 'enUS', 'FB', 'accepts', 'xxXX', 'var', 'msgloadmore', 'Load', 'More', 'var', 'pagemode', 'screen', 'var', 'dialogtitle', 'var', 'dialogmessage', 'The', 'Beacon', 'Second', 'Reader', 'James', 'H', 'Fassett', 'Free', 'Ebook', 'qrcode', 'background', 'transparent', 'url', 'norepeat', 'Project', 'Gutenberg', 'free', 'ebooks', 'The', 'Beacon', 'Second', 'Reader', 'James', 'H', 'Fassett', 'No', 'cover', 'available', 'Download', 'Bibrec', 'Bibliographic', 'Record', 'Author', 'Fassett', 'James', 'H', 'James', 'Hiram', 'Title', 'The', 'Beacon', 'Second', 'Reader', 'Language', 'English', 'LoC', 'Class', 'PE', 'Language', 'Literatures', 'English', 'Subject', 'Fairy', 'tales', 'Subject', 'Readers', 'Subject', 'Reading', 'Elementary', 'Category', 'Text', 'EBookNo